In https://www.biorxiv.org/content/10.1101/2024.01.06.574462v1
Hirano et al. show that CAP peptides might play a role in the formation of at least some plant galls. 
On multiple sequence alignment they found that CAP sequences from insect and plant host had highly conserveed 6 AA and 22 AA regions (designated CAP-p6 and CAP-p22).


In this notebook I show a quick example of using regex within Neo4j to find these conserveed residues within protein sequences. I also show the time difference between a search that first finds proteins annotated by PFAM's "PF00188" (Cysteine-rich secretory protein family) HMM model then performs the regex filter dramatically speeds up the search compared to the regex filter alone. 

Lastly, antiSMASH regions containing CAP proteins are exported into a clustermap.js plot.

This is all done with a connection to the SocialGene RefSeq database.


In [4]:
from socialgene.neo4j.neo4j import GraphDriver 
from socialgene.config import env_vars
from socialgene.base.socialgene import SocialGene
from socialgene.clustermap.serialize import SerializeToClustermap
from socialgene.search.hmmer import SearchDomains
from pathlib import Path
import pandas as pd
env_vars["NEO4J_URI"] = "bolt://localhost:7687"

In [5]:
%%time
# Just showing that the connection is to the database containing 343,381 RefSeq genomes
with GraphDriver() as db:
    results = db.run(
        """
            MATCH (a1:assembly)
            WHERE a1.uid STARTS WITH "GCF"
            RETURN count(a1) as assembly_count
        """
    ).to_df()

results

CPU times: user 2.56 ms, sys: 368 µs, total: 2.93 ms
Wall time: 53.1 ms


,assembly_count
0,343381


In [6]:
%%time
with GraphDriver() as db:
    results = db.run(
        """
            MATCH (n:pfam)<-[:SOURCE_DB]-(h1:hmm)
            WHERE n.acc starts with "PF00188"
            MATCH (h1)-[:ANNOTATES]->(p1:protein)<-[:ENCODES]-(n1:nucleotide)-[:ASSEMBLES_TO]-(a1:assembly)
            RETURN count(DISTINCT a1) as assembly_count
        """
    ).to_df()
results


CPU times: user 3.02 ms, sys: 0 ns, total: 3.02 ms
Wall time: 1.19 s


,assembly_count
0,135595


In [13]:
%%time
with GraphDriver() as db:
    results = db.run(
        """
            MATCH (n:pfam)<-[:SOURCE_DB]-(h1:hmm)
            WHERE n.acc starts with "PF00188"
            MATCH (h1)-[:ANNOTATES]->(p1:protein)<-[e1:ENCODES]-(n1:nucleotide)-[:ASSEMBLES_TO]-(a1:assembly)
            WHERE not e1.antismash_region is null
            RETURN count(DISTINCT a1) as assembly_count
        """
    ).to_df()
results


CPU times: user 3.1 ms, sys: 0 ns, total: 3.1 ms
Wall time: 2.65 s


,assembly_count
0,5593


In [7]:
%%time
with GraphDriver() as db:
    results = db.run(
        """
            MATCH (n:pfam)<-[:SOURCE_DB]-(h1:hmm)
            WHERE n.acc starts with "PF00188"
            MATCH (h1)-[:ANNOTATES]->(p1:protein)<-[:ENCODES]-(n1:nucleotide)-[:ASSEMBLES_TO]-(a1:assembly)
            WHERE p1.sequence =~ '.*[FY]TQ[IV]VW.*'
            RETURN count(DISTINCT a1) as assembly_count
        """
    ).to_df()
results


CPU times: user 2.46 ms, sys: 45 µs, total: 2.51 ms
Wall time: 898 ms


,assembly_count
0,2337


In [8]:
%%time
with GraphDriver() as db:
    results = db.run(
        """
            MATCH (n:pfam)<-[:SOURCE_DB]-(h1:hmm)
            WHERE n.acc starts with "PF00188"
            MATCH (h1)-[:ANNOTATES]->(p1:protein)<-[e1:ENCODES]-(n1:nucleotide)-[:ASSEMBLES_TO]-(a1:assembly)
            WHERE p1.sequence =~ '.*[FY]TQ[IV]VW.*' and not e1.antismash_region is null
            RETURN count(DISTINCT a1) as assembly_count
        """
    ).to_df()
results


CPU times: user 0 ns, sys: 3.32 ms, total: 3.32 ms
Wall time: 881 ms


,assembly_count
0,43


Now pull all the genomic regions and make a clinker plot

In [9]:
%%time
with GraphDriver() as db:
    df = db.run(
        """
        MATCH (n:pfam)<-[:SOURCE_DB]-(h1:hmm)
        WHERE n.acc starts with "PF00188"
        MATCH (h1)-[:ANNOTATES]->(p1:protein)<-[e1:ENCODES]-(n1:nucleotide)-[:ASSEMBLES_TO]-(a1:assembly)
        WHERE p1.sequence =~ '.*[FY]TQ[IV]VW.*' and not e1.antismash_region is null
        MATCH (:protein)<-[e2:ENCODES]-(n1)
        WHERE e2.antismash_region = e1.antismash_region
        RETURN n1.uid as uid, e1.antismash_region, min(e2.start) as start, max(e2.end) as end, collect(DISTINCT p1.uid) as cap_protein_uids
           
        """
    ).to_df()

CPU times: user 3.94 ms, sys: 1.35 ms, total: 5.29 ms
Wall time: 959 ms


In [7]:
%time
sg=SocialGene()
for line in df.iterrows():
    sg.fill_given_locus_range(locus_uid=line[1].uid, start=float(line[1].start), end=float(line[1].end))
    

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.77 µs


In [8]:
for assembly in sg.assemblies.values():
    for locus in assembly.loci.values():
            locus.add_bgcs_by_feature(features=locus.features)

In [9]:
# Not calculating any links
link_df = pd.DataFrame(columns=['query_feature', 'target_feature'])
group_df = pd.DataFrame(columns=['query_feature', 'target_feature'])

z = SerializeToClustermap(
        sg_object=sg,
        sorted_bgcs=sg.assemblies.values(),
        link_df=link_df,
        group_df=group_df,
    )

Get the genes that were discovered as CAP proteins in the database search, so they can be highlighted in the clustermap plot

In [10]:
caps = []
cap_protein_uids = {x for xs in df.cap_protein_uids.to_list() for x in xs}
for k1,v1 in sg.assemblies.items():
    for k2,v2 in v1.loci.items():
        for feature in v2.features:
            if feature.protein.uid in cap_protein_uids:
                caps.append(feature)

In [11]:
group_df = pd.DataFrame({
    'query_feature': [caps[0]] * len(caps),
    'target_feature': caps
})

z = SerializeToClustermap(
        sg_object=sg,
        sorted_bgcs=sg.assemblies.values(),
        link_df=link_df,
        group_df=group_df,
    )


z.write("clustermap/data.json")

2024-07-16 18:20:06 INFO     Writing clustermap.js output to: clustermap/data.json                                                    ]8;id=847870;file:///home/chase/Documents/github/kwan_lab/socialgene/sgpy/socialgene/clustermap/serialize.py\serialize.py]8;;\:]8;id=302671;file:///home/chase/Documents/github/kwan_lab/socialgene/sgpy/socialgene/clustermap/serialize.py#140\140]8;;\

                    INFO     Creating clustermap.js clusters                                                                           ]8;id=505813;file:///home/chase/Documents/github/kwan_lab/socialgene/sgpy/socialgene/clustermap/serialize.py\serialize.py]8;;\:]8;id=124651;file:///home/chase/Documents/github/kwan_lab/socialgene/sgpy/socialgene/clustermap/serialize.py#40\40]8;;\

                    INFO     Creating clustermap.js links                                                                             ]8;id=680675;file:///home/chase/Documents/github/kwan_lab/socialgene/sgpy/socialgene/clustermap/serialize.py\serialize.py]8;;\:]8;id=894479;file:///home/chase/Documents/github/kwan_lab/socialgene/sgpy/socialgene/clustermap/serialize.py#117\117]8;;\

                    INFO     Creating clustermap.js links                                                                              ]8;id=509076;file:///home/chase/Documents/github/kwan_lab/socialgene/sgpy/socialgene/clustermap/serialize.py\serialize.py]8;;\:]8;id=749538;file:///home/chase/Documents/github/kwan_lab/socialgene/sgpy/socialgene/clustermap/serialize.py#92\92]8;;\